In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
# 用來呼叫瀏覽器
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的，用來下指令
import os

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''

In [ ]:
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options ) #跑了之後會開一個無痕視窗，會有寫「Chrome 目前受到自動測試軟體控制」

# driver.set_window_size(1200, 960) #視窗大小設定 (寬，高)
# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化

# 放置爬取的資料
listData = []

'''
以 function 名稱，作為爬蟲流程
'''

# 走訪頁面
def visit():
    driver.get('https://www.104.com.tw/jobs/main/');  #get: 把網址寫在瀏覽器
    
# 輸入關鍵字
def search():
    # 輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, "input#ikeyword")  # find_element: 找元素，此處為找搜尋列
    # 單一元素:find_element  多元素: find_elements  
    # 可在youtube的搜尋列按右鍵選檢查，可看到一些id之類的東西
    # CSS: id --> #    Class --> .
    # input#search: 告訴電腦我input要寫id為search
    txtInput.send_keys("Python")  # send_keys: 幫你打字，也可用來輸帳號密碼、寫文章之類的
    sleep(1)  #要等他，不然後面會有error!
    
    # 選擇台北、新北 
    areaInput = driver.find_element(By.CSS_SELECTOR, 'span#icity[name="icity"][data-value="地區"]')
    areaInput.click()
    sleep(1)  #要等他!!!不然下面地區會找不到!!!
    
    TpeInput = driver.find_element(By.CSS_SELECTOR, 'input[value="6001001000"]') # 台北
    TpeInput.click() 
    #sleep(1)
    
    NewTpeInput = driver.find_element(By.CSS_SELECTOR, 'input[value="6001002000"]') # 新北
    NewTpeInput.click() 
    #sleep(1)
    
    btnInput = driver.find_element(By.CSS_SELECTOR, "button.category-picker-btn-primary") #按下送出
    btnInput.click()
    sleep(1)
    
    # 按下搜尋
    driver.find_element(By.CSS_SELECTOR, 'button[class="btn btn-primary js-formCheck"]').click()
    sleep(1)
    
    # 按下「全職」
    driver.find_element(By.CSS_SELECTOR, 'div.b-float-left li[data-value="1"]').click()
    sleep(0.5)
    
def scroll():
    # 瀏覽器內部的高度，當你一直滾、快滾到底的時候會更新
    innerHeightOfWindow = 0
    
    # 當前捲動的量(高度)
    totalOffset = 0
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while totalOffset <= innerHeightOfWindow:
        # 每次移動高度
        totalOffset += 300;
        
        # 捲動的 js code  #下面這段是java script的程式
        js_scroll = '''(
            function (){{
                window.scrollTo({{
                    top:{}, 
                    behavior: 'smooth' 
                }});
            }})();'''.format(totalOffset)
        # 每次會滾到top後面{}的值，並透過.format(totalOffset)把值放到{}裡面
        # behavior: "smooth" 表示平滑的滑動，而不是一次跳到那個位置
        
        # 執行 js code
        driver.execute_script(js_scroll)
        
        # 強制等待
        sleep(1)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeightOfWindow = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        );
        
        # 強制等待
        sleep(0.5)
        
        # 印出捲動距離
        print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))
        
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if totalOffset >= 1000:
            break
    
def parse():
    articles = driver.find_elements(By.CSS_SELECTOR, 'div#js-job-content article')
        
    for article in articles:
        # 印出分隔文字
        print("=" * 30)
        
        # 職缺名稱、職缺連結
        job = article.find_element(By.CSS_SELECTOR, 'a.js-job-link')
        job_title = job.text
        job_link = job.get_attribute("href")
        print(job_title, job_link)
        
        # 公司名稱、公司連結
        company = article.find_element(By.CSS_SELECTOR, 'div.b-block__left li a')
        company_title = company.text
        company_link = company.get_attribute("href")
        print(company_title, company_link)
        
        # 地區
        place = article.find_element(By.CSS_SELECTOR, 'ul.b-list-inline.b-clearfix.job-list-intro.b-content li').text
        print(place)
        
        # 放資料到list中
        listData.append({
            "job_title": job_title,
            "job_link": job_link,
            "company_title": company_title,
            "company_link": company_link,
            "place": place
        })

# 將 list 存成 json
def saveJson():
    fp = open("job_104.json", "w", encoding = "utf-8") # 開一個新檔案叫"youtube.json"，w for write、寫入資料
    fp.write(json.dumps(listData, ensure_ascii = False))
    fp.close

# 關閉瀏覽器
def close():
    driver.quit()
    
# 主程式
if __name__ == '__main__':
    visit()
    search()
    scroll()
    parse()
    saveJson()
    close()
    print("done")